In [ ]:
from numpy import infty, zeros
from math import *
from __future__ import annotations
import graphviz as gv
from typing import Any

class Graph:

  #Inicializa las listas de vértices y aristas.
  def __init__( self ):
    self._vertices: list[Graph.Vertex] = []
    self._edges: list[Graph.Edge] = []
    self.identifier = 'Code' #Define un identificador predeterminado para los vértices.

  # Devuelve una representación en cadena del grafo.
  def __repr__( self ): return f"Graph: ( \n \t Vertices: { self._vertices }, \n \t Edges: { self._edges } \n)"

  @property
  #Devuelve una copia de la lista de vértices
  def vertices( self ) -> list[Graph.Vertex] : return self._vertices.copy()

  @property
  #Devuelve una lista de diccionarios que representan los datos de los vértices.
  def data(self) -> list[dict] : return [ v.data for v in self._vertices ]

  @property
  #Devuelve una copia de la lista de aristas.
  def edges( self ) -> list[Graph.Edge] : return self._edges.copy()

  @property
  #Calcula y devuelve una matriz de costos entre los vértices.
  def costMatrix( self ) -> list[list[int]]:
    vertexNumber = len(self._vertices)
    matrix = [ [ 0 for i in range( vertexNumber ) ] for j in range( vertexNumber ) ]
    for edge in self.edges:
      i, j = self.vertices.index( edge.vertices[0] ), self._vertices.index( edge.vertices[1] )
      matrix[i][j] = matrix[j][i] = edge.weight
    return matrix

  class Vertex:
    #Inicializa los datos del vértice, su identificador y una lista de aristas conectadas a él.
    def __init__( self, data ):
      self.data: dict = data
      self.identifier = 'Code'
      self.edges: list[Graph.Edge] = []

    def __repr__( self ):
      return str( self.data[self.identifier] )

    #Implementa la comparación de menor que para vértices.
    def __lt__( self, vertex ): # <
      for i in vertex.edges:
        if i.vertices[1] == self: return True
      return False

    def __gt__( self, vertex ): # >
      for i in self.edges:
        if i.vertices[1] == vertex: return True
      return False

    def __le__( self, vertex ): # <=
      return self < vertex

    def __ge__( self, vertex ): # =>
      return self > vertex

    def __eq__( self, compare ):
      if isinstance( compare, Graph.Vertex ) : return self.data == compare.data
      elif isinstance( compare, dict )       : return self.data == compare
      elif isinstance( compare, int )        : return self.data[ self.identifier ] == compare
      return False

    def __hash__(self):
      return hash(str(self.data[self.identifier]))

  class Edge:

    #Inicializa los vértices de origen y destino y el peso de la arista.
    def __init__( self, fromVertex: Graph.Vertex, toVertex: Graph.Vertex, weight ):
      self.vertices: tuple(Graph.Vertex,Graph.Vertex) = ( fromVertex, toVertex )
      self.weight = weight

    def __repr__( self ):
      return f"( {self.weight}:{self.vertices[0]} <-> {self.vertices[1]} )"

  #Agrega un nuevo vértice al grafo.
  def newVertex( self, data ):
    self._vertices.append( self.Vertex( data ) )

  #Agrega una nueva arista entre dos vértices con un peso dado.
  def newEdge( self, source: str, destination: str, weight: float ):

    source = self.getVertex(source)
    destination = self.getVertex(destination)

    edge = self.Edge( source, destination, weight )

    source.edges.append( edge )
    destination.edges.append( self.Edge( destination, source, weight ) )

    self._edges.append( edge )

  #Obteniene un vértice específico según su valor.
  def getVertex( self, value ):
    left = 0 #índice izquierdo al comienzo de la lista de vértices.
    right = len( self.vertices ) - 1 #índice derecho al final de la lista de vértices.

    #Búsqueda binaria para encontrar el vértice
    while left <= right:
        mid = left + (right - left) // 2 #índice medio del rango actual.
        vertex = self.vertices[mid].data[self.identifier]
        if vertex == value: return self.vertices[mid]
        elif vertex < value: left = mid + 1
        else: right = mid - 1
    return None

  def addVertex( self, vertex ):
    self._vertices.append( vertex )


  #Calcula la distancia entre dos vértices en coordenadas geográficas.
  def distance( self, source, destination ):

    source = self.getVertex( source )
    destination = self.getVertex( destination )

    # Radio medio de la Tierra en kilómetros
    earth_radius = 6371

    # Convierte las coordenadas de grados a radianes
    lt0 = radians(source.data['Latitude'])
    ln0 = radians(source.data['Longitude'])
    ltf = radians(destination.data['Latitude'])
    lnf = radians(destination.data['Longitude'])

    # Diferencia de latitud y longitud.
    dlt = ltf - lt0
    dln = lnf - ln0

    # Fórmula de Haversine.
    a = sin( dlt / 2 )**2 + cos(lt0) * cos(ltf) * sin(dln / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return earth_radius * c # Distancia en kilómetros.


  #Obtiene una lista de rutas en el grafo.
  def getRoutes( self ):
    routes = []
    for e in self.edges:
      s = e.vertices[0].data
      d = e.vertices[1].data
      route = {
          'Source': ( s['Latitude'], s['Longitude'] ),
          'Destination': ( d['Latitude'], d['Longitude'] ),
          'Weight': e.weight
          }
      routes.append(route)
    return routes

  #Algoritmo de Dijkstra para encontrar el camino mínimo.
  def dijskstra( self, start ):

    paths = { v: ( infty, None ) for v in self.vertices } #Diccionario para mantener las distancias más cortas y los predecesores.
    paths[start] = ( 0, None ) #Distancia desde el vértice de inicio a sí mismo como 0 y sin predecesor.

    toVisit = self.vertices #Lista con todos los vértices para visitar.
    toVisit.remove( start ) #Elimina el vértice de inicio para que no se visite nuevamente.

    for e in start.edges:
      paths[ e.vertices[1] ] = ( e.weight, start )

    while toVisit:
      toVisit = sorted( toVisit, key = lambda x: paths[x] ) #Ordena la lista según distancias actuales.
      min = toVisit.pop(0) #Vértice con la distancia más corta del vértice de inicio.

      #Recorre las aristas saliende de min.
      for e in min.edges:
        if paths[min][0] + e.weight < paths[ e.vertices[1] ][0]:
          paths[ e.vertices[1] ] = ( paths[min][0] + e.weight, min ) #Actualiza la distancia y el predecesor

    return(paths)

  #Determina el camino mínimo entre dos vértices.
  def takePath( self, minpaths, source, destination ):

    #Comprueba si el destino no está en 'minpaths' o si su distancia es infinita.
    if not destination in minpaths or minpaths[destination] == ( infty, None ):
      return f"{source} -/-> {destination} "

    weight = minpaths[destination][0] #Peso del camino más corto desde el inicio hasta el destino.
    path = f"{destination}" #Inicializa una cadena 'path' con el vértice de destino.

    #Mientras el vértice actual no sea igual al vértice de inicio.
    while not source == destination:
      destination = minpaths[destination][1] #Obtiene el predecesor del vértice actual.
      path = f"{destination} -> " + path #Agrega el vértice actual al camino.

    #Construye y devuelve una cadena que representa el camino más corto.
    return f"({round(weight,3)}): " + path

  #Visualización del mapa.
  def display( self ):
    plot = gv.Graph( comment = "Graph", engine = 'sfdp' )
    for vertex in self.vertices:
      plot.node( str(vertex) )
    for edge in self.edges:
      plot.edge( str(edge.vertices[0]), str(edge.vertices[1]), label = str(round(edge.weight,3)) )
    return plot

In [ ]:
from pandas import read_csv

data = read_csv('/content/flights_final.csv')

#Crea dos listas vacías para almacenar datos de aeropuertos y rutas.
airports = []
routes = []

for i, row in data.iterrows():

  source = {} #Diccionario para los datos de origen del aeropuerto.
  destination = {} #Diccionario para los datos de destino del aeropuerto.

  for item in dict(row):

    if 'Source' in item: source[ item.replace('Source Airport ', '') ] = row[item]
    elif 'Destination' in item: destination[ item.replace('Destination Airport ', '') ] = row[item]

  if not source in airports: airports.append( source ) #Si 'source' no está en 'airports', agrégalo a la lista de aeropuertos.
  if not destination in airports: airports.append( destination ) #Si 'destination' no está en 'airports', agrégalo a la lista de aeropuertos.

  #Crea una tupla 'route' que representa una ruta entre aeropuertos.

  route = (source['Code'],destination['Code'])
  if not ( route in routes or route[::-1] in routes ): routes.append(route)

  #Limita la cantidad de registros procesados a 500 para acelerar la ejecución.
  #if i > 500: break

#Ordena la lista de aeropuertos según el código del aeropuerto.
airports = sorted(airports, key=lambda x: x['Code'])

#Representar el grafo de rutas entre aeropuertos.
globe = Graph()

#Agrega cada aeropuerto como un vértice al grafo.
for airport in airports:
  globe.newVertex( airport )

#Itera sobre las rutas y agrega cada ruta como una arista con su distancia calculada.
for route in routes:
  source = globe.getVertex(route[0])
  destination = globe.getVertex(route[1])
  globe.newEdge( route[0], route[1], globe.distance(route[0], route[1]) )


In [ ]:
globe = Graph()

for airport in airports:
  globe.newVertex( airport )

for route in routes:
  source = globe.getVertex(route[0]) #Obtiene los vértices de origen y destino del grafo correspondientes a la ruta.
  destination = globe.getVertex(route[1])
  globe.newEdge( route[0], route[1], globe.distance(route[0], route[1]) ) # Agrega una nueva arista(ruta), con peso igual a distancia entre aeropuertos.


In [ ]:
import folium #Librería de Python para crear mapas interactivos.

#Centrar el mapa en Colombia.
map = folium.Map()

airports = globe.data
routes = globe.getRoutes()

for airport in airports:
  latitud = airport['Latitude']
  longitud = airport['Longitude']
  nombre_lugar = ''
  for info in airport:
    nombre_lugar += f"{info}: {airport[info]} \n"

  #Colocar marcadores sobre el mapa utilizando la latitud y longitud. Se muestra el título al presionar.
  folium.Marker([latitud, longitud], popup=nombre_lugar).add_to(map)

for route in routes:
  folium.PolyLine( (route['Source'],route['Destination']) , color="red").add_to(map)

#Guardar el mapa en un archivo html.
map.save('mapa_Global.html')

In [ ]:
# Primer item
vts = None
while not vts:
  vts = globe.getVertex(input('Digite el código del aeropuerto: ').upper())
  print('')

for info in vts.data: #Imprime información sobre el aeropuerto de origen (vértice 'vts').
  print(f"{info}: {vts.data[info]}")
print()

minpaths = globe.dijskstra(vts) #Caminos mínimos desde el aeropuerto de origen

#Filtra y ordena los caminos mínimos para mostrar primero los más cortos.
minpaths = {
    key: minpaths[key]
    for key in sorted( minpaths, key=lambda x: minpaths[x][0], reverse = True)
    if not minpaths[key][0] == infty
  }

i = 0
#Itera sobre los caminos mínimos y muestra información relacionada con cada camino.
for airport in minpaths:

  #Calcula y almacena el camino mínimo desde 'vts' al aeropuerto actual.
  cam = globe.takePath( minpaths, vts, airport )
  caminitos = cam[cam.index('(') + 1:cam.index(')')].split(" -> ")

  #Muestra el camino mínimo y la distancia.
  print(globe.takePath( minpaths, vts, airport))
  print('Distancia es :', float(caminitos[0]))
  print()

  for info in airport.data: #Imprime información sobre el aeropuerto de destino (airport).
    print(f"{info}: {airport.data[info]}")
  print()

  #Limita la impresión a los primeros 10 caminos mínimos.
  i += 1
  if i >= 10: break

Digite el código del aeropuerto: fra

Code: FRA
Name: Frankfurt am Main Airport
City: Frankfurt
Country: Germany
Latitude: 50.033333
Longitude: 8.570556

(37438.885): FRA -> BEG -> TXL -> PEK -> IST -> TIA -> VCE -> BCN -> OTP -> MAD -> MIA -> CUN -> DEN -> CLE -> TPA
Distancia es : 37438.885

Code: TPA
Name: Tampa International Airport
City: Tampa
Country: United States
Latitude: 27.97550011
Longitude: -82.53320313

(36650.539): FRA -> BEG -> TXL -> PEK -> IST -> TIA -> VCE -> BCN -> OTP -> MAD -> MIA -> ASU
Distancia es : 36650.539

Code: ASU
Name: Silvio Pettirossi International Airport
City: Asuncion
Country: Paraguay
Latitude: -25.23999977
Longitude: -57.52000046

(36548.555): FRA -> BEG -> TXL -> PEK -> IST -> TIA -> VCE -> BCN -> OTP -> MAD -> MIA -> CUN -> DEN -> CLE -> TTN
Distancia es : 36548.555

Code: TTN
Name: Trenton Mercer Airport
City: Trenton
Country: United States
Latitude: 40.27669907
Longitude: -74.81349945

(35943.556): FRA -> BEG -> TXL -> PEK -> IST -> TIA -> VCE

In [ ]:
vtf = None
while not vtf:
  vtf = globe.getVertex(input('Digite el código del aeropuerto: ').upper())
  print('')

for info in vtf.data:
  print(f"{info}: {vtf.data[info]}")

print()
print(globe.takePath( minpaths, vts, vtf ))

ruta_minima = globe.takePath(minpaths, vts, vtf)  # Obtiene la ruta mínima como una cadena formateada
codigo_aeropuertos = []  # Inicializa una lista para almacenar los códigos de aeropuertos

# Procesa la cadena de la ruta mínima y extrae los códigos de aeropuertos
if '(' in ruta_minima:
    ruta_minima = ruta_minima[ruta_minima.index('): ') + 3:]  # Recorta la parte antes de los códigos
    codigo_aeropuertos = ruta_minima.split(' -> ')  # Divide la cadena en códigos individuales

Digite el código del aeropuerto: tpa

Code: TPA
Name: Tampa International Airport
City: Tampa
Country: United States
Latitude: 27.97550011
Longitude: -82.53320313

(37438.885): FRA -> BEG -> TXL -> PEK -> IST -> TIA -> VCE -> BCN -> OTP -> MAD -> MIA -> CUN -> DEN -> CLE -> TPA


In [ ]:
import folium

# Crea un nuevo mapa centrado en la ubicación inicial
map = folium.Map(location=[20, 0], zoom_start=2)


location_ini = None
for codigo in codigo_aeropuertos:
    for airport in airports:
        if airport['Code'] == codigo:
            latitud = airport['Latitude']
            longitud = airport['Longitude']
            nombre_lugar = ''
            for info in airport:
                nombre_lugar += f"{info}: {airport[info]} \n"

            # Coloca un marcador en el mapa para el aeropuerto con latitud y longitud
            folium.Marker([latitud, longitud], popup=nombre_lugar).add_to(map)

            location_actual = (latitud, longitud)

            if location_ini is not None:
                folium.PolyLine([location_ini, location_actual], color="blue").add_to(map)

            # Establece el punto actual como el punto de inicio para el próximo segmento
            location_ini = location_actual

# Guarda el mapa en un archivo HTML
map.save('mapa_ruta_minimo.html')